In [1]:
import AFQ.data as afqd
import cloudknot as ck
import importlib
import itertools
import os
import s3fs
import bids

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credenti

In [2]:
ck.set_region('us-west-2')

## Define the preprocessing function

In [3]:
def recon_hbn_subject(subject):
    import AFQ.data as afqd
    import os
    import json
    import subprocess
    from s3fs import S3FileSystem
    
    msmt_recon = {
      "name": "mrtrix_multishell_msmt_1M",
      "space": "T1w",
      "atlases": [
        "schaefer100x7",
        "schaefer100x17",
        "schaefer200x7",
        "schaefer200x17",
        "schaefer400x7",
        "schaefer400x17",
        "brainnetome246",
        "aicha384",
        "gordon333",
        "aal116",
        "power264"
      ],
      "anatomical": [
        "mrtrix_5tt"
      ],
      "nodes": [
        {
          "name": "msmt_csd",
          "software": "MRTrix3",
          "action": "csd",
          "output_suffix": "msmtcsd",
          "input": "qsiprep",
          "parameters": {
            "mtnormalize": True,
            "response": {
              "algorithm": "dhollander"
            },
            "fod": {
              "algorithm": "msmt_csd",
              "max_sh": [
                4,
                8,
                8
              ]
            }
          }
        },
        {
          "name": "track_ifod2",
          "software": "MRTrix3",
          "action": "tractography",
          "output_suffix": "ifod2",
          "input": "msmt_csd",
          "parameters": {
            "use_5tt": True,
            "use_sift2": True,
            "tckgen": {
              "algorithm": "iFOD2",
              "select": 2000000,
              "max_length": 250,
              "min_length": 30,
              "power": 0.33,
              "crop_at_gmwmi": True,
              "backtrack": True
            },
            "sift2": {}
          }
        },
        {
          "name": "mrtrix_conn",
          "software": "MRTrix3",
          "action": "connectivity",
          "output_suffix": "msmtconnectome",
          "input": "track_ifod2",
          "parameters": {
            "tck2connectome": [
              {
                "zero_diagonal": False,
                "search_radius": 2,
                "scale_invnodevol": True,
                "symmetric": True,
                "use_sift_weights": True,
                "stat_edge": "sum",
                "measure": "sift_invnodevol_radius2_count"
              },
              {
                "zero_diagonal": False,
                "search_radius": 2,
                "scale_invnodevol": False,
                "symmetric": True,
                "length_scale": "length",
                "use_sift_weights": False,
                "stat_edge": "mean",
                "measure": "radius2_meanlength"
              },
              {
                "zero_diagonal": False,
                "search_radius": 2,
                "scale_invnodevol": False,
                "symmetric": True,
                "use_sift_weights": False,
                "stat_edge": "sum",
                "measure": "radius2_count"
              },
              {
                "zero_diagonal": False,
                "search_radius": 2,
                "scale_invnodevol": False,
                "symmetric": True,
                "use_sift_weights": True,
                "stat_edge": "sum",
                "measure": "sift_radius2_count"
              }
            ]
          }
        }
      ]
    }

    with open("./msmt.json", "w") as fp:
        json.dump(msmt_recon, fp)
        
    local_bids_folder = "hbn"
    local_output_folder = "hbn-recon-output"

    input_bucket = "fcp-indi"
    input_s3_prefix = "data/Projects/HBN/BIDS_curated"

    output_bucket = "hbn-afq"
    output_s3_prefix = ""
    
    # select subjects from qsiprep
    study = afqd.S3BIDSStudy(
        "hbn-curated",
        input_bucket,
        input_s3_prefix,
        subjects=[subject],
        random_seed=42
    )

    study.download(local_bids_folder, include_derivs=True)
                
    response = subprocess.run(
        [
            "qsiprep",
            os.path.abspath(os.path.join(".", local_bids_folder)),
            os.path.abspath(os.path.join(".", local_output_folder)),
            "participant",
            "--recon-input",
            os.path.abspath(os.path.join(".", local_bids_folder, "derivatives", "qsiprep")),
            "--recon-spec",
            os.path.abspath("./msmt.json"),
            "--recon_only",
            "--participant_label",
            subject,
            "--nthreads",
            "8",
            "--omp-nthreads",
            "8",
            "-w",
            "./scratch",
        ],
        check=True
    )
    
    output_dir = "/".join([output_bucket, "derivatives"])

    print("putting " + subject + " in s3 bucket " + output_dir + "/")
    fs = S3FileSystem()
    fs.put(os.path.abspath(os.path.join(".", local_output_folder)),
           output_dir + "/",
           recursive=True)

    print("successfully put " + subject + " in s3 bucket " + output_dir + "/")

    return {subject: True}

# Create a cloudknot DockerImage instance

where the base_image is a customized version of qsiprep
This version of the qsiprep docker image is located on AWS ECR with image URI:
[454929164628.dkr.ecr.us-east-1.amazonaws.com/qsiprep:nrdg](https://console.aws.amazon.com/ecr/repositories/qsiprep/?region=us-east-1)

In [4]:
recover_from_config = True

In [5]:
if not recover_from_config:
    di = ck.DockerImage(
        name="recon-hbn-subject-0",
        func=recon_hbn_subject,
        base_image="qsiprep:direct-0.12.1",
        github_installs=["https://github.com/yeatmanlab/pyAFQ.git@master",
                         "https://github.com/matplotlib/matplotlib.git@v2.2.3",
                         "https://github.com/bids-standard/pybids.git@0.9.3"],
        overwrite=True,
    )
else:
    di = ck.DockerImage("recon-hbn-subject-0")

## Build, tag, and push the Docker image

In [6]:
if not recover_from_config:
    di.build(tags=["hbn-recon-20210126"])

In [7]:
if not recover_from_config:
    repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())

In [8]:
if not recover_from_config:
    # The very first time you run this, this command could take
    # a few hours because the docker image is large
    di.push(repo=repo)

In [9]:
print(di.repo_uri)

454929164628.dkr.ecr.us-west-2.amazonaws.com/cloudknot:hbn-recon-20210126


## Create the Knot

In [10]:
# Specify bid_percentage to use Spot instances
# And make sure the volume size is large enough. 55-60 GB seems about right for HBN preprocessing. YMMV.
# knot = ck.Knot(
#     name=f"hbn-recon-20210126-2",
#     docker_image=di,
#     pars_policies=("AmazonS3FullAccess",),
#     bid_percentage=100,
#     memory=64000,
#     job_def_vcpus=8,
#     max_vcpus=8192,
#     volume_size=90,
#     retries=1,
#     aws_resource_tags={"Project": "HBN-FCP-INDI"},
# )

knot = ck.Knot(name=f"hbn-recon-20210126-2")

## Submit the jobs and check on results

In [11]:
import AFQ.data as afqd
study = afqd.S3BIDSStudy(
    "hbn-curated",
    "fcp-indi",
    "data/Projects/HBN/BIDS_curated/derivatives/qsiprep",
)

subjects = study._all_subjects
print(len(subjects))

Retrieving subject S3 keys
[########################################] | 100% Completed |  0.6s
2136


In [12]:
recon_study = afqd.S3BIDSStudy(
    "hbn-recon",
    bucket="hbn-afq",
    s3_prefix="derivatives/qsirecon",
    subjects="all",
    anon=False,
)

Retrieving subject S3 keys
[########################################] | 100% Completed |  3min 17.0s


In [13]:
subs_missing_recon = list(set(study._all_subjects) - set(recon_study._all_subjects))

In [14]:
len(subs_missing_recon)

43

In [15]:
results = knot.map(subs_missing_recon)

In [16]:
ck.set_region('us-west-2')
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
6ec51e04-f267-44d3-abd8-0b92773731f8        hbn-recon-20210126-2-2        SUBMITTED
c9cb2323-03b0-48cd-b295-a1d0f92a7d8e        hbn-recon-20210126-2-1        FAILED   
f586aa44-981a-43cc-855f-9e8f6ee15c2a        hbn-recon-20210126-2-0        FAILED   


## Results

The results are dicts where the keys are the subject IDs and the values report sucess or failure

In [ ]:
results.result()

## When you're done, clobber the knot

In [ ]:
knot.clobber(clobber_pars=True)